# Settings

In [ ]:
import pathlib

dir_data = pathlib.Path("../data/docs")

# ETL

In [ ]:
from hlm12rag.etl import etl_data_from_kaggle

etl_data_from_kaggle(dataset="rtatman/questionanswer-dataset", dst=dir_data)